# Lecture 20 – Data 100, Spring 2024

Data 100, Spring 2024

[Acknowledgments Page](https://ds100.org/sp24/acks/)

## Starting Up SQL

Before we look at SQL syntax in detail, let's first get ourselves set up to run SQL queries in Jupyter.

### Approach #1: SQL Magic

**1. Load the `sql` Module.** 

Load `%%sql` cell magic.

In [ ]:
%load_ext sql

**2. Connect to a database.**  

Here, we connect to the SQLite database `basic_examples.db`.

In [ ]:
%sql duckdb:///data/example_duck.db --alias duck

If you were connecting to an "enterprise data platform"

```python
from sqlalchemy import create_engine

snow_engine = create_engine(
    f"snowflake://{user}:{password}@{account_identifier}")
%sql snow_engine --alias snow

db_engine = create_engine(
  url = f"databricks://token:{access_token}@{server_hostname}?" +
        f"http_path={http_path}&catalog={catalog}&schema={schema}"
)
%sql db_engine --alias db
```

<br/>

**3. Run a simple SQL query.** 

Note the `%%sql` lets Jupyter parse the rest of the lines as a SQL command.

In [ ]:
%%sql
SELECT * FROM Dragon;

**Simple query, this time on two different lines.**

In [ ]:
%%sql
SELECT *
FROM Dragon;

### Approach #3: `pd.read_sql`

It turns out that `pandas` has a special-purpose function to parse SQL queries. We can pass in a SQL query as a string to return a `pandas` DataFrame. To achieve the same result as we did using cell magic above, we can do the following.

**1. Connect to a database**

In [ ]:
import sqlalchemy 
import pandas as pd

engine = sqlalchemy.create_engine("duckdb:///data/example_duck.db")


**2. Run a simple SQL query**

In [ ]:
query = """
SELECT * 
FROM Dragon;
"""

df = pd.read_sql(query, engine)
df

### Approach "3" -- Duck DB Special

Now that we are using DuckDB we can do something extra crazy:

In [ ]:
import seaborn as sns
mpg = sns.load_dataset("mpg")

In [ ]:
%%sql
SELECT * FROM mpg

That is right!! DuckDB can also see my dataframes in the python environment allowing me to do dataframe manipulation in SQL!

<br><br>
**Return to Lecture**
<br><br><br>


---

## Tables and Schema

A **database** contains a collection of SQL **tables**. Let's connect to our "toy" database `basic_examples.db` and explore the tables it stores.

In [ ]:
%%sql
SELECT * FROM information_schema.tables

In [ ]:
%%sql
SELECT * FROM information_schema.columns

### Getting Schema information with SQLAlchemy 
How you list the tables varies across database platforms.  For example, the statement:

```sql
SELECT * FROM information_schema.columns
```

only works on Postgres compatible databases.

For example, if we wanted to get the schema for tables in sqlite we would need the following:

In [ ]:
pd.read_sql("SELECT * FROM sqlite_schema", "sqlite:///data/basic_examples.db")

Fortunately, SQLAlchemy has some generic tools that will be helpful regardless of what database platform you use.

In [ ]:
from sqlalchemy import inspect
inspector = inspect(engine)
inspector.get_table_names()

In [ ]:
inspector.get_columns('scene')

Same with SQLite

In [ ]:
sqlite_engine = sqlalchemy.create_engine("sqlite:///data/basic_examples.db")
inspect(sqlite_engine).get_columns("scene")


Example of table creation with interesting constraints

More advanced example of creating tables with primary and foreign key constraints:

In [ ]:
%%sql

DROP TABLE IF EXISTS grade;
DROP TABLE IF EXISTS assignment;
DROP TABLE IF EXISTS student;


CREATE TABLE student (
    student_id INTEGER PRIMARY KEY,
    name VARCHAR,
    email VARCHAR
);

CREATE TABLE assignment (
    assignment_id INTEGER PRIMARY KEY,
    description VARCHAR
);

CREATE TABLE grade (
    student_id INTEGER,
    assignment_id INTEGER,
    score REAL CHECK (score > 0 AND score <= 100),
    FOREIGN KEY (student_id) REFERENCES student(student_id),
    FOREIGN KEY (assignment_id) REFERENCES assignment(assignment_id)
);

INSERT INTO student VALUES
(123, 'JoeyG', 'jegonzal@berkeley.edu'),
(456, 'NargesN', 'norouzi@berkeley.edu');

INSERT INTO assignment VALUES
(1, 'easy assignment'),
(2, 'hard assignment');



In [ ]:
%%sql 
INSERT INTO grade VALUES
(123, 1, 80),
(123, 2, 42),
(456, 2, 100);



In [ ]:
%sql SELECT * FROM grade;

<br/><br/><br/>

---

## Basic Queries

Every SQL query *must* contain a `SELECT` and `FROM` clause.

* `SELECT`: specify the column(s) to return in the output
* `FROM`: specify the database table from which to extract data

In [ ]:
%%sql
SELECT * FROM Dragon;

In [ ]:
%%sql
SELECT cute, year FROM Dragon;

**Aliasing** with `AS`

In [ ]:
%%sql
SELECT cute AS cuteness,
       year AS "birth year"
FROM Dragon;

**Uniqueness** with `DISTINCT`

In [ ]:
%%sql
SELECT DISTINCT year
FROM Dragon;

**Filtering** with `WHERE`

In [ ]:
%%sql
SELECT name, year
FROM Dragon
WHERE cute > 0;

In [ ]:
%%sql
SELECT name, cute, year
FROM Dragon
WHERE cute > 0 OR year > 2013;

In [ ]:
%%sql
SELECT name, year
FROM Dragon 
WHERE name IN ('puff', 'hiccup');

In [ ]:
%%sql
SELECT name, cute
FROM Dragon
WHERE cute IS NOT NULL;

**Ordering** data using `ORDER BY`

In [ ]:
%%sql
SELECT *
FROM Dragon
ORDER BY cute DESC;

**Restricting** output with `LIMIT` and `OFFSET`

In [ ]:
%%sql
SELECT *
FROM Dragon
LIMIT 2;

In [ ]:
%%sql
SELECT *
FROM Dragon
LIMIT 2
OFFSET 1;

## Grouping Data with `GROUP BY`

In [ ]:
%%sql
SELECT *
FROM Dish;

A small note: the fact that `type` is highlighted in green below is a consequence of Jupyter assuming that we are writing Python code (where `type` is a built-in keyword). `type` does *not* have a special meaning in SQL, so the color below does not indicate any special functionality. When we run the cell, Jupyter realizes it should recognize the code as SQL. 

In [ ]:
%%sql
SELECT type
FROM Dish;

In [ ]:
%%sql
SELECT type
FROM Dish
GROUP BY type;

In [ ]:
%%sql
SELECT type, SUM(cost)
FROM Dish
GROUP BY type;

In [ ]:
%%sql
SELECT type, 
       SUM(cost), 
       MIN(cost),
       MAX(name)
FROM Dish
GROUP BY type;

In [ ]:
%%sql
SELECT year, COUNT(cute)
FROM Dragon
GROUP BY year;

In [ ]:
%%sql
SELECT year, COUNT(*)
FROM Dragon
GROUP BY year;

## Working with the `sql` results as Python variables

By default, executing a query with a magic command produces output but doesn't save it into any Python variable:

In [ ]:
%sql SELECT * FROM Dragon

This can be inconvenient if you later want to do further processing of these data in Python.

### Storing one-line `%sql` queries

For simple one-line queries, you can use IPython's ability to store the result of a magic command like `%sql` as if it were any other Python statement, and save the output to a variable:

In [ ]:
dragon_table = %sql SELECT * FROM Dragon
dragon_table

As noted above, the result of the query is a Python variable of type `ResultSet`, more specifically:

In [ ]:
type(dragon_table)

You need to manually convert it to a Pandas DataFrame if you want to do pandas-things with its content:

In [ ]:
dragon_df = dragon_table.DataFrame()
dragon_df

You can configure `jupysql` to _automatically_ convert all outputs to Pandas DataFrames. This can be handy if you intend all your Python-side work to be done with Pandas, as it saves you from manually having to call `.DataFrame()` first on all outputs. On the other hand, you don't get access to the original SQL `ResultSet` object, which have a number of interesting properties and capabilities. You can learn more about those in the [jupysql documentation](https://jupysql.ploomber.io).

For now, let's turn this on so you can see how this simplified, "pandas all the way" worfklow looks like:

In [ ]:
%config SqlMagic.autopandas = True

In [ ]:
dragon_df = %sql SELECT * FROM Dragon
dragon_df

In [ ]:
type(dragon_df)

### Storing multi-line `%%sql` queries

For a more complex query that won't fit in one line, such as for example:

In [ ]:
%%sql
SELECT year, COUNT(*)
FROM Dragon
GROUP BY year;

You can use the `variable <<` syntax in jupysql to store its output (this will honor your `autopandas` state and store either a `sql.run.ResultState` or a Pandas `DataFrame`):

In [ ]:
%%sql dragon_years <<
SELECT year, COUNT(*)
FROM Dragon
GROUP BY year;

In [ ]:
dragon_years

## More Adavnced SQL with DuckDB

You could do a lot what we do in this class just using Duck DB and the many duck db functions:

https://duckdb.org/docs/sql/functions/overview

In [ ]:
from ds100_utils import fetch_and_cache

In [ ]:
url = "https://gist.github.com/domoritz/fd517a3a3a210c24a488e61870e2cf2c/raw/b1d53719e8e0eb9f6a95de82fdaccf0b001c0dea/flights-1m.parquet"
fetch_and_cache(url,"flights.parquet")

In [ ]:
%%sql
SELECT * FROM 'data/flights.parquet' LIMIT 10;

In [ ]:
%%sql avg_delays <<

SELECT 
    dayname(fl_date) AS "Day of the Week", 
    mean(dep_delay) AS "Mean Departure Delay", 
FROM 'data/flights.parquet'
GROUP BY "Day of the Week"

In [ ]:
import plotly.express as px
px.bar(avg_delays, x="Day of the Week", y = "Mean Departure Delay",
       category_orders={"Day of the Week": ["Sunday", "Monday", "Tuesday", 
                      "Wednesday", "Thursday", "Friday", "Saturday"]})